### Building Masked Language Model Dataset.

In this notebook , we will query our database , get the data and then build the dataset for the Masked language model. 

To achieve this we will leverage hugging face tranformers for Name entity recogniton, we will train the model on our new article.

The data is comming as a list of of document , we will split each document into sentences or paragraphs.

For each sentence we will run the NER model and find the entities.

For each entity found in a sentence , we will consider the sentence with the masked entity as a question and the entity we have masked as the answer and we will save our dataset in a list for further steps.

In [1]:
from config import DATABASE_URL
from src.utils.database_connection_utils import get_database_session

In [2]:
session, engine = get_database_session(DATABASE_URL)

In [3]:
sql_query = "select * from article where website_origin not in ('https://www.voalingala.com/', 'https://www.karismatv.cd')"

In [4]:
import pandas as pd
import numpy as np

In [5]:
data = None
with engine.connect() as connection:
    data = pd.read_sql_query(sql=sql_query, con=connection)

In [8]:
from pathlib import Path
DATA_PATH = Path.cwd().joinpath("data")
assert DATA_PATH.exists(), "the data path does not exist"


In [6]:
data.shape

(140638, 9)

In [11]:
data_file_path = DATA_PATH.joinpath("corpus", "raw", 'drc-news-raws.csv')

In [9]:
data.loc[:, ["content", "posted_at"]].to_csv(data_file_path, index=False, header=False)

In [14]:
data = pd.read_csv(data_file_path, names=["content", "posted_at"], index_col=False)

In [18]:
data = data.sort_values(by=["posted_at"], ascending=False)

In [19]:
data.head()

,content,posted_at
78,Le juge Dieudonné Kamuleta n’a pas remplacé Di...,2022-12-05 00:00:00
19,Le principal opposant au régime de Félix Tshis...,2022-12-05 00:00:00
77,"L’ancien Premier ministre, Augustin Matata Pon...",2022-12-05 00:00:00
76,Le Sénat déclare recevables les propositions d...,2022-12-05 00:00:00
141,Excellence Monsieur le Président et co-fondate...,2022-12-05 00:00:00


In [20]:
from gensim.utils import deaccent

In [22]:
from nltk.tokenize import sent_tokenize

In [34]:
import re

In [45]:
def remove_accents(input_str):
    input_without_accent = deaccent(input_str)
    return input_without_accent

In [46]:
def tokenize_document(document):
    """given a document split the document into sentences and return a list of those sentence

    Args:
        document (_type_): _description_

    Returns:
        _type_: _description_
    """
    tokenized_document = sent_tokenize(document, language="french")
    return tokenized_document

def replace_point(document):
    """replace the point with the wwt.www with space point before tokenizing the document .

    Args:
        document (_type_): _description_
    """
    result = re.sub(r"(\S)\.(\S)", r"\1 . \2", document)
    return result

In [47]:
def pre_process_document(document):
    """pre-process the document by removing the accents, replacing the point and tokenizing the document.

    Args:
        document (_type_): _description_
    """
    document = remove_accents(document)
    document = replace_point(document)
    document = tokenize_document(document)
    return document

In [51]:
random_index = np.random.randint(0, data.shape[0])
sample_doc = data.content[random_index]
print(pre_process_document(sample_doc))

['La guerre fait rage entre les deux Msr.', 'Chaque camp revendique le label du parti.', 'Le Msr pro-pouvoir a indique mercredi qu’il a recu l’agrement du ministere de l’Interieur et menace de traduire en justice ceux qui l’utiliseront abusivement.', 'Le Mouvement social pour le renouveau reste un et indivisible et toujours loyal a son initiateur, le president Joseph Kabila, a indique le depute Francois Rubota.', 'Selon lui, le parti qui evoluait depuis 2005 sous couvert de l’arrete ministeriel du MNR, a finalement eu son propre agrement.', 'Rubota rappelle que le Msr n’a jamais bascule a l’opposition.', 'Il appelle le G7 a revoir sa charte et a ne considerer que six partis et non sept comme l’indique certaines personnes.', 'Entre-temps, le Msr pro-opposition est monte au creneau.', 'Sa direction politique a denonce les strategies de destabilisation du parti par des mains noires.', 'Ils ont appele les militants de leur parti a la resistance pour faire echec a cet arrete ministeriel, pe

In [40]:
result = re.sub(r"(\S)\.(\S)", r"\1 . \2", sample_doc)

In [41]:
print(result)

La Dynamique pour la Sauvegarde et la Consolidation des Acquis Démocratiques a échangé le week-end dernier avec des jeunes  des confessions religieuses des communes de Mont-Ngafula et Selembao autour du thème : "Comment consolider et sauvegarder les acquis démocratiques ?". Ce thème a permis de faire un état de lieu sur le fonctionnement des institutions actuelles en République Démocratique du Congo . Au regard de divergences sur les lois en ce qui concerne les élections constatées dans le chef de certains politiques, le conférencier du jour, le professeur Auguste Mampuya, a invité l'assistance à ne pas se laisser distraire par ces opinions d'intérêts égoïstes . "Il y en a qui veulent toucher à tel aspect, il y en a qui veulent toucher à un tel autre aspect, tout simplement parce que chacun veut que les lois servent ses propres intérêts", a déclaré le professeur Auguste Mampuya . Ce dernier a appelé à une vraie unité pour permettre la mise en place des réformes totales et profondes de 

In [52]:
from nltk.corpus.reader.util import concat
from gensim.corpora.csvcorpus import CsvCorpus
from gensim.utils import deaccent

In [53]:
import logging
import csv
import itertools
from gensim import utils
logger = logging.getLogger(__name__)

In [54]:
import json

Once we have loaded our dataset , the next step is to leverage the HuggingFace transformer to train the model. to find the NER.

In [55]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner-with-dates")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner-with-dates")

In [56]:
from transformers import pipeline

nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")

In [57]:
from secrets import token_hex

In [72]:
from tqdm import tqdm

In [149]:
import difflib

In [107]:
drc_new_corpus_file_with_masks = DATA_PATH.joinpath("interim", "drc-news-with-masked.csv")

In [108]:
assert drc_new_corpus_file_with_masks.parent.exists(), "the corpus file does not exist"

In [109]:
input_drc_new_corpus_file = DATA_PATH.joinpath("corpus", "raw", "drc-news-raws.csv")

In [110]:
assert input_drc_new_corpus_file.exists()

In [174]:
drc_news_corpus = DrcNewsCorpus(input_drc_new_corpus_file, False)

In [175]:
drc_news_corpus.save_question_answer_to_file(drc_new_corpus_file_with_masks, n_sample=None)

363434it [84:19:09,  3.82it/s]                                  

: 

: 